#  Topic Models with tomotopy 
With this notebook you can load a corpus from a zip file and create topic models using the Python package tomotopy. Tomotopy is a Python extension of *tomoto* (*to*pic *mo*deling *to*ol), which is a [Gibbs-sampling](https://www.youtube.com/watch?v=BaM1uiCpj_E) based topic model library written in C++. 

Read more in the tomotopy documentation... 
## <img src=https://bab2min.github.io/tomotopy/tomoto.png align="left" width="20">[**tomotopy**](https://bab2min.github.io/tomotopy/v0.12.2/en/)

Work through the notebook. The key things to do are:
1. ensure you know how to upload and unzip datafiles;
2. understand what the model parameters are doing and try different values to see the effect they have on the topics returned;
3. try training some different size models (e.g. 10 topics, 30 topics, 50 topics);  
4. explore the topic for documents and assess the quality of topics returned; 
5. measure topic coherence for a number of models using the different coherence metrics;
6. infer topics for an unseen document and find similar documents based on the topic distributions;
7. make notes on your observations of different models and the kinds of similarities between documents they produce.

Since we need to evaluate topic models against a use case - consider the idea of a recommendation engine: what model performs best for recommending similar items and why?

**Important:** Each time you change settings, you will need to re-run the cells that create the topic modelling pipeline.

<div style="border:1px solid black;margin-top:1em;padding:0.5em;">
    <strong>Task 0:</strong> Throughout the notebook there are defined tasks for you to do. Watch out for them - they will have a box around them like this! Make sure you take some notes as you go.
</div>

## Import the necessary Python packages

In [ ]:
# various packages
from zipfile import ZipFile
import os.path
from os import path
import glob
import re
from pathlib import Path
from importlib import reload
# from collections import Counter, Iterable
import datetime

# topic modeling / nlp packages
import tomotopy as tp

# visualisation / exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
pyLDAvis.enable_notebook()
from IPython.display import IFrame, Markdown, display
from scipy.spatial import distance

# These last lines of code suppress deprecation warnings displaying in the notebook
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Install Python packages if needed

After running the cell above you may see an import error telling you that you are missing certain packages such as tomotopy or seaborn. If this is the case, uncomment the relevant packages in the cell below, run the cell, and then re-run the cell above.

In [ ]:
# !pip install tomotopy
# !pip install seaborn

**Note:** if you get an error with stopwords later in the notebook, uncomment the first two lines in the cell below and re-run... 

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

## Define functions

The following cells contain functions to load a corpus from a directory of text files and preprocess the corpus.

In [ ]:
def load_data_from_dir(path):
    file_list = glob.glob(path + '/*.txt')

    # create document list:
    documents_list = []
    source_list = []
    for filename in file_list:
        with open(filename, 'r', encoding='utf8') as f:
            text = f.read()
            f.close()
            if len(text.split()) > 100: # only keep docs longer than 100 words
                documents_list.append(text)
                source_list.append(os.path.basename(filename))
    print(f"Total Number of Documents in '{path}':",len(documents_list))
    
    return documents_list, source_list

In [ ]:
def preprocess_data(doc_set, extra_stopwords={}):
    # adapted from https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python
    
    # replace all newlines or multiple sequences of spaces with a standard space
    doc_set = [re.sub(r'\s+', ' ', doc) for doc in doc_set]
    
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    
    # add any extra stopwords
    if (len(extra_stopwords) > 0):
        en_stop = en_stop.union(extra_stopwords)
    
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # add tokens to list
        texts.append(stopped_tokens)
    
    return texts

## Unzip a corpus

Upload the zip file containing your corpus of text files to the same directory as this notebook. The corpus_filename variable below must match the name of your zip file. Then run the following cell to unzip it.

In [ ]:
corpus_filename = 'transcripts2008-2018.zip' # change to the name of your dataset zip file

# Create a ZipFile Object and load sample.zip in it
with ZipFile(corpus_filename, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

## Load and pre-process the corpus
Load the corpus and preprocess with additional stop words (if required).

The name of the directory that contains the extracted corpus text files should be assigned to the 'corpus_dirname' variable below. Make sure the only text files in this directory are those belonging to the corpus.

In [ ]:
corpus_dirname = 'transcripts2008-2018'

# adjust the path below to wherever you have the transcripts2018 folder
documents_list, source_list = load_data_from_dir(corpus_dirname)

# You can add extra stopwords here, between the curly brackets, in addition to NLTK's stopword list
doc_clean = preprocess_data(documents_list, {'applause', 'laughter'})

## Set parameters
In the cell below you can set the parameters of the LDA topic model. More information about the tomotopy LDA model parameters can be found [here](https://bab2min.github.io/tomotopy/v0.4.1/en/#tomotopy.LDAModel).

* α – alpha, a Dirichlet prior on the per-document topic distribution
* β – beta / eta, a Dirichlet prior on the per-topic word distribution
* optim_interval - how often to optimise the beta hyperparameter
* k – the number of topics in the model
* burn-in – the number of burn-in iterations
* iter – the number of iterations


<div style="border:1px solid black;margin-top:1em;padding:0.5em;">
    <strong>Task 1:</strong> Read about the hyperparameters (settings) used in the tomotopy LDA topic model and consider the effect you expect each one to have on the results. Experiment with models using different settings. The settings will be displayed in your model summary; you can copy this into a Word doc or text file to keep a record of the settings you have tried. 
    Write a new combination of settings you tried in the cell below.
</div>

_Your answer here..._

In [ ]:
# Minimum frequency of words (integer)
# Words with a smaller document frequency than min_df are excluded from the model
# Default is 0 - i.e. no words are excluded
# For more info see https://bab2min.github.io/tomotopy/v0.12.3/en/#vocabulary-controlling-using-cf-and-df
min_doc_freq = 5

# Number of top words to be removed (integer)
# Setting this to 1 or more removes common words from the model
# Default is 0 - i.e. no words are excluded
remove_top = 10

# Number of topics to return, between 1 and 32767
num_topics = 20

# You can read more about the following alpha and beta hyperparameters here:
# https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d

# Alpha
# Hyperparameter of Dirichlet distribution for document-topic
# Controls the density of topics per document
# a float
doc_topic = 0.1

# Beta
# Hyperparameter of Dirichlet distribution for topic-word
# Note this is 'eta' in tomotopy - it's not a typo!
# Controls the density of words per topic
# a float
topic_word = 0.01

# Set the burn-in
# Number of initial iterations that are discarded before optimising hyperparameters
# This speeds up the convergence of the model on an optimal set of topics
brn_in = 10

# Number of iterations of the Gibbs sampler
# If we specify 20 here, we will run 200 (10*20) iterations of Gibbs sampling total in the training loop
num_iterations = 20

# Set the top n words from the topic to display in the output of results
num_topic_words = 10

## Train the model and display the results
Run the cell below to train the model and display the results (you shouldn't need to change anything here).

<div style="border:1px solid black;margin-top:1em;padding:0.5em;">
    <strong>Task 2:</strong> Look through the output and discuss with a small group of classmates. Consider the impact of the different parameter settings on the results. There are several ways to optimise and tweak the results. Try to:
    
- test more or fewer topics.
- test more or fewer number of iterations
- test the `min_doc_freq` and `remove_top` variables
    
Which parameter settings produce the clearest topic groupings?
</div>

_Your answers here..._

In [ ]:
# Adapted from https://bab2min.github.io/tomotopy/v0.12.2/en/

# Intialize the model

# The default term weighting is used
model = tp.LDAModel(tw=tp.TermWeight.ONE,
                    min_df=min_doc_freq, 
                    rm_top=remove_top, 
                    k=num_topics, 
                    alpha=doc_topic, 
                    eta=topic_word
                   )

model.burn_in = brn_in

# Add each document to the model
for text in doc_clean:
    model.add_doc(text)

print("Topic Model Training...\n")

# train the model
# the loop reports LL/word every 10 iterations
# this is a measure of model fit to the data (higher is better)
for i in range(0, 100, 10):
    model.train(iter=num_iterations)

topics = []
topic_individual_words = []
for topic_number in range(0, num_topics):
    topic_words = ' '.join(word for word, prob in model.get_topic_words(topic_id=topic_number, top_n=num_topic_words))
    print(f'\nTop 10 words of topic #{topic_number}\n')
    print(model.get_topic_words(topic_id=topic_number, top_n=num_topic_words))
    print()
    topics.append(topic_words)
    topic_individual_words.append(topic_words.split())
    
    
print("\nModel Summary\n")
model.summary()

In [ ]:
# Print the frequent words removed by rm_top. Do we want these in the model?
model.removed_top_words

## Visualise the topic model

The topic model can be visualised using the [pyLDAvis](https://pyldavis.readthedocs.io/en/latest/index.html) Python library, which is adapted from the R package developed by Carson Sievert and Kenny Shirley (see link to the original paper below).

The left side of the pyLDAvis chart shows us the distribution of the topics in our model, and how closely they are related. The right side of the chart shows us the most relevant terms for the selected topic. Decreasing the value of lambda using the slider on the top right lets you re-rank the words displayed to be more specific to your topic of interest. This can make it easier to see what the topic is describing. Hovering over a word will display the other topics the word appears in on the left side of the chart. You can watch a video [here](https://www.youtube.com/watch?v=IksL96ls4o0) that explains how to interpret the LDAvis chart in much more detail.

Read more:
[Sievert, C., & Shirley, K. E. (2014). LDAvis: A method for visualizing and interpreting topics. Proceedings of the Workshop on Interactive Language Learning, Visualization, and Interfaces (pp. 63-70). Baltimore: Association for Computational Linguistics.](http://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf)

**Important note:** the pyLDAvis topic numbers will be off by one from the results in our model summary (i.e., Topic 1 in pyLDAvis will be Topic 0 from the tomotopy output above). 

In [ ]:
# Adapted from https://github.com/bab2min/tomotopy/blob/main/examples/lda_visualization.py
# Run this cell, then look for the 'ldavis.html' file in your Jupyterhub directory - open it in a new tab

warnings.filterwarnings("ignore", category=DeprecationWarning) 

topic_term_dists = np.stack([model.get_topic_word_dist(k) for k in range(model.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in model.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in model.docs])
vocab = list(model.used_vocabs)
term_frequency = model.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency,
    sort_topics=False # IMPORTANT: otherwise the topic_ids between pyLDAvis and tomotopy are not matching!
)
pyLDAvis.save_html(prepared_data, 'ldavis.html')

In [ ]:
# View the html visualisation
# You can view it here but it's better to open the html file in a new browser tab
warnings.filterwarnings("ignore", category=DeprecationWarning) 

IFrame(src='ldavis.html', width=900, height=900)

<div style="border:1px solid black;margin-top:1em;padding:0.5em;">
    <strong>Task 3:</strong> Take five minutes to explore the interactive visualisation and try different values of &lambda; (lambda). This allows you examine words in topics along a scale, weighted entirely by the probability of the word given the topic if &lambda; = 1, or weighted by a ratio of this to the probability (ie frequency) of the word in the whole corpus. What value of &lambda; filters the topics best? Are there good / bad topics here? Write down some examples.
</div>

_Your answer here..._

## Examine top documents for a given topic
The following code to display the top documents is adapated from [***Topic Modeling - With Tomotopy***](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/09-Topic-Modeling-Without-Mallet.html) from the book *Introduction to Cultural Analytics & Python* by Melanie Walsh (2021).

In [ ]:
topic_distributions = [list(doc.get_topic_dist()) for doc in model.docs]

topic_indiv_150_words = []
for topic_number in range(0, num_topics):
    topic_words_150 = ' '.join(word for word, prob in model.get_topic_words(topic_id=topic_number, top_n=150))
    topic_indiv_150_words.append(topic_words_150.split())

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

def make_md(string):
    display(Markdown(str(string)))

def get_top_docs(docs, sources, topic_indiv_150_words, topic_distributions, topic_index, n):
    
    sorted_data = sorted([(_distribution[topic_index], _document, _sources) 
                          for _distribution, _document, _sources
                          in zip(topic_distributions, docs, sources)], reverse=True)
    
    top_25 = ", ".join(topic_indiv_150_words[topic_index][:25])
    make_md\
    (f"### Topic {topic_index}\n\
    \n{top_25} ...\
    \n\n**Note**: words from the top 150 topic words are shown in bold in the text\n\n---")
    
    for probability, doc, source in sorted_data[:n]:
        # Make topic words bolded
        for word in topic_indiv_150_words[topic_index]:
            doc = doc.lower()
            if word in doc:
                doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc)
                #doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc, re.IGNORECASE)
        
        make_md(f'  \n**Topic Probability**: {probability}  \n**Source**: {source}  \n**Document**: {doc}  \n\n---')
    
    return

In [ ]:
# Choose the topic number to explore
topic_no = 3

# Set the number of 'top docs' to display
num_top_docs = 5

In [ ]:
# Display top documents for the selected topic, with topic words highlighted

get_top_docs(documents_list, 
             source_list,
             topic_indiv_150_words, 
             topic_distributions, 
             topic_index = topic_no, 
             n = num_top_docs)

<div style="border:1px solid black;margin-top:1em;padding:0.5em;">
    <strong>Task 4:</strong> Choose a topic and write a couple of sentences below that describe what the topic is about, noting words that clearly fit together and any odd words that do not seem to fit.
</div>

_Your description of a topic here..._

## Infer topics for an unseen document

In [ ]:
# First we need to unzip a folder of new transcripts
unseen_corpus_filename = 'ted-transcripts2019.zip' # change to the name of your dataset zip file

# Create a ZipFile Object and load sample.zip in it
with ZipFile(unseen_corpus_filename, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall(path='transcripts2019')

In [ ]:
# Select the document for which you want to infer the topics
new_doc_path = 'transcripts2019/transcripts2019/2019-01-22-iseult_gillespie_why_should_you_read_fahrenheit_451.txt'

In [ ]:
# preprocess the new document and add it to the model, ready to infer topics
with open(new_doc_path, 'r', encoding='utf8') as f:
    new_text = f.read()

new_text_clean = preprocess_data([new_text], {'applause', 'laughter'})
new_doc = model.make_doc(new_text_clean[0])

print(new_text[:1000])

In [ ]:
# infer topics in the new doc

new_doc_topics = model.infer(new_doc)

new_doc_list = []

for topic, prop in enumerate(new_doc_topics[0].tolist()):
    new_doc_list.append((topic,prop))
    
new_doc_topics_sorted = sorted(new_doc_list, key=lambda x: x[1], reverse=True) # sorts document topics

In [ ]:
# display topic proportions and words for the new document

for topic, prop in new_doc_topics_sorted:
    topic_words = [word for word, prop in model.get_topic_words(topic_id=topic, top_n=num_topic_words)]
    print("%.3f" % prop, topic, topic_words)

## Find similar documents

We can documents with a similar topic distribution using Jensen-Shannon distance. A lower distance score indicates documents have more similar topics.

For more information, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jensenshannon.html

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

jenshan_scores = []

# here topic_distributions from the 'Examine top documents' section are used
for doc, distribution in enumerate(topic_distributions):
    a = new_doc_topics[0].tolist()
    b = distribution
    jenshan_score = distance.jensenshannon(a, b)
    jenshan_scores.append((doc,jenshan_score))

In [ ]:
dist_index = sorted(jenshan_scores, key=lambda x: x[1]) # The smaller the Jensen-Shannon distance, the more similar the docs

print('Similar documents by Jensen-Shannon distance (lower is better):\n')

for i in range(3): 
    document_id, js_score = dist_index[i]
    print('---------',document_id, js_score,'---------')
    print(re.sub('\s+', ' ', documents_list[document_id][:2000]))
    print()

    doc_list = []
    for i, text in enumerate(documents_list):
        doc_list.append((i, text))
    
    document_topics = enumerate(topic_distributions[document_id]) 
    document_topics = sorted(document_topics, key=lambda x: x[1], reverse=True) # sorts document topics
    
    for topic, prop in document_topics:
        topic_words = [word for word, prop in model.get_topic_words(topic_id=topic, top_n=num_topic_words)]
        if prop > 0.05:
            print ("%.3f" % prop, topic, topic_words)
    
    print()

## Get a coherence score
Topic coherence scores attempt to measure how similar the prominent words in a topic are to each other and, as a result, how easy the topic is for a human to interpret. Some of the better topic coherence metrics are based on mutual information, a collocation measure we've seen already. You don't need to study these in detail for this course, but if you're curious can read more about them in [Roder et al (2015)](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf), [Lau et al (2014)](https://aclanthology.org/E14-1056.pdf) and nice summary [here](https://palmetto.demos.dice-research.org/) that explains the c_uci score: 

>"c_uci is a coherence that is based on a sliding window and the pointwise mutual information (PMI) of all word pairs of the given top words. The word cooccurrence counts are derived using a sliding window with the size 10. For every word pair the PMI is calculated. The arithmetic mean of the PMI values is the result of this coherence. c_npmi is an enhanced version of the c_uci coherence using the normalized pointwise mutual information (NPMI) instead of the pointwise mutual information (PMI)."

The code below uses the tomotopy presets to calculate the average coherence for the model and the coherence per topic. There are various coherence metrics available:
* u_mass
* c_uci
* c_npmi (we'll use this as an example. It builds on c_uci.)
* c_v (not recommended anymore - the authors have stated there's a problem with it)
* c_p (not available through tomotopy, but through the palmetto library or webservice). 

These scores can be used to assess whether changes in the number of topics or other hyperparameters improves the model when you have large numbers of topics. However, it is very important to also use qualitative assessment, since in many cases coherence scores are not as reliable as human judgment.

The next code block calculates and prints a c_npmi coherence score for each topic in our current model, as well as an average score for the model as a whole.

In [ ]:
# Calculate coherence scores for each topic
coh = tp.coherence.Coherence(model, coherence='c_npmi')
average_coherence = coh.get_score()
coherence_per_topic = [coh.get_score(topic_id=k) for k in range(model.k)]

# Print the average coherence score
print('==== Coherence : c_npmi ====')
print(f'Average: {average_coherence:.6f}')

# Print the topic number along with its coherence score
print('Per Topic:')
for topic_idx, coherence_score in enumerate(coherence_per_topic):
    print(f'Topic {topic_idx}: {coherence_score:.6f}')
print()

To visualise the variation in the per-topic coherence scores we could use a box & whisker plot.

In [ ]:
coh_per_topic_df = pd.DataFrame(coherence_per_topic)
coh_per_topic_df.plot.box(title='c_npmi coherence scores for our ' + str(model.k) + ' topics in this model')
plt.show()

## Calculate coherence scores for models with different numbers of topics

The code below creates further models below with different numbers of topics and return the **c_npmi** coherence score for each of them. The other parameters remain as you set them at the start of the notebook.

NPMI scores will range from 1 to -1. For a single pair of topic words the score can be interpreted as follows:

* 1 = the words only co-occur in a sliding window together, never independently (positive collocation)
* 0 = the words co-occur with the same probability as the product of their independent probabilities (null / no collocation)
* -1 = the words never co-occur in a sliding window together, and only appear independently (negative collocation, more likely not to appear together than expected!)

Higher scores indicate greater coherence for this measure. We generally want to compare two or more scores for different topics or models.

If you want a bit more detail, you can read about how normalise pointwise mutual information is calculated in section 3.1 of [Bouma 2009](https://svn.spraakdata.gu.se/repos/gerlof/pub/www/Docs/npmi-pfd.pdf) who is Lau et al's source for this measure.

## Test a range of topic sizes and plot the results
**Important**: training a lot of models with larger corpora could take a long time. Experiment with small numbers of models before going large, so as not to be waiting too long! 

In [ ]:
# supply values for k and the interval
# eg with min_k=5, max_k=100, and intervals=5 will train models with 5, 10, 15, 20 ... 100 topics.
min_k = 5
max_k = 50
intervals = 5

coherences = {}

for i in range(min_k, max_k + 1, intervals):   
    lda_mod = tp.LDAModel(tw=tp.TermWeight.ONE,
                        min_df = min_doc_freq, 
                        rm_top = remove_top, 
                        k = i, 
                        alpha = doc_topic, 
                        eta = topic_word
                       )

    lda_mod.burn_in = brn_in

    # Add each document to the model
    for text in doc_clean:
        lda_mod.add_doc(text)

    lda_mod.train(iter=num_iterations)

    coherences[i] = tp.coherence.Coherence(lda_mod, coherence = 'c_npmi').get_score()

In [ ]:
# convert the coherence scores to a pandas dataframe
df = pd.DataFrame.from_dict(coherences, orient='index', columns=['Coherence'])
df['Topics'] = df.index

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning) 

# plot the result
df.plot(kind='line', x='Topics', y='Coherence', ylim=(0,0))
plt.show()

<div style="border:1px solid black;margin-top:1em;padding:0.5em;">
    <strong>Task 4:</strong> What number of topics produces the best coherence score? How does this compare with your qualitative assessment of topic quality? Write about your findings below.
</div>

_Your findings here..._